In [1]:
import pandas as pd
import numpy as np

data = pd.read_csv('Results/RequiredDataSet.csv',header=None, names=['booking_date','channel','property_id','room_name','room_id','check_in','check_out','room_rate','booking_status'])
# data.head()


In [2]:
Results = pd.read_excel('Results/DataSetWithStayDate.xlsx')
del Results['Old_Room_rate']
# Results.head()

In [3]:
scrapyData = pd.read_csv('Results/eventData2015.csv',header=None, names=['Day','Date','Event','HolidayIn'])
scrapeResult = scrapyData.dropna()
# scrapeResult.head()

In [4]:
# to avoid "SettingWithCopyWarning"
pd.options.mode.chained_assignment = None  # default='warn'

In [5]:
scrapeResult['Date'] =   '2015-' + scrapeResult['Date'] 
# scrapeResult.head()

In [6]:
scrapeResult['Date'] = pd.to_datetime(scrapeResult.Date)
# scrapeResult.head()

In [7]:
del scrapeResult['HolidayIn']
del scrapeResult['Day']
# scrapeResult.head()

In [8]:
newResults = Results.filter(['Property_id','Stay_date','Booking_status','Room_rate'], axis=1)
# newResults.head()

In [9]:
newResults = newResults[(newResults.Booking_status == 'COMPLETED') | (newResults.Booking_status == 'CONFIRMED')]
# newResults

In [10]:
newResults = newResults[(newResults.Stay_date >= '2015-01-01') & (newResults.Stay_date <= '2015-12-31')]
# newResults.head()

In [11]:
newResults['Stay_date'] = pd.to_datetime(newResults.Stay_date)
# newResults.head()

In [12]:
res = pd.merge(left=newResults, left_on='Stay_date',right=scrapeResult, right_on='Date')
# res1 = res.drop_duplicates()
res1 = res.reset_index()
del res1['index']
del res1['Date']
# res1.head()

In [13]:
res1['room_count'] = res1.groupby(['Property_id','Stay_date'])['Room_rate'].transform('count')
res2 = res1.drop_duplicates()
res2.head()

,Property_id,Stay_date,Booking_status,Room_rate,Event,room_count
0,BB,2015-01-01,COMPLETED,39500.00,New Years Day,26
2,KR,2015-01-01,COMPLETED,19387.25,New Years Day,12
3,SS,2015-01-01,COMPLETED,20130.00,New Years Day,23
4,KK,2015-01-01,COMPLETED,447.35,New Years Day,13
5,SS,2015-01-01,COMPLETED,31008.66,New Years Day,23


In [14]:
res2['totalRevenue'] = res2.groupby(['Property_id','Stay_date'])['Room_rate'].transform('sum')
res3 = res2.drop_duplicates()
res3.head()

,Property_id,Stay_date,Booking_status,Room_rate,Event,room_count,totalRevenue
0,BB,2015-01-01,COMPLETED,39500.00,New Years Day,26,226229.66
2,KR,2015-01-01,COMPLETED,19387.25,New Years Day,12,170272.65
3,SS,2015-01-01,COMPLETED,20130.00,New Years Day,23,293631.32
4,KK,2015-01-01,COMPLETED,447.35,New Years Day,13,53759.07
5,SS,2015-01-01,COMPLETED,31008.66,New Years Day,23,293631.32


In [15]:
res3['ADR'] = res3.apply(lambda res3: (res3['totalRevenue'] / res3['room_count'] ) , axis=1)
res4 =res3
res4.head()

,Property_id,Stay_date,Booking_status,Room_rate,Event,room_count,totalRevenue,ADR
0,BB,2015-01-01,COMPLETED,39500.00,New Years Day,26,226229.66,8701.140769
2,KR,2015-01-01,COMPLETED,19387.25,New Years Day,12,170272.65,14189.387500
3,SS,2015-01-01,COMPLETED,20130.00,New Years Day,23,293631.32,12766.579130
4,KK,2015-01-01,COMPLETED,447.35,New Years Day,13,53759.07,4135.313077
5,SS,2015-01-01,COMPLETED,31008.66,New Years Day,23,293631.32,12766.579130


In [16]:
# len(res4[res4.channel=="OTA"])
del res4['Booking_status']
del res4['Room_rate']
res5 = res4.drop_duplicates()
# res5

In [18]:
res5.head()

,Property_id,Stay_date,Event,room_count,totalRevenue,ADR
0,BB,2015-01-01,New Years Day,26,226229.660000,8701.140769
2,KR,2015-01-01,New Years Day,12,170272.650000,14189.387500
3,SS,2015-01-01,New Years Day,23,293631.320000,12766.579130
4,KK,2015-01-01,New Years Day,13,53759.070000,4135.313077
9,MB,2015-01-01,New Years Day,62,422873.764848,6820.544594


In [28]:
temp = res5['Property_id'].unique()
type(temp)

numpy.ndarray

In [29]:
for i in range(len(temp)):
    property_id = res5[(res5.Property_id == temp[i])]
    property_id.to_excel('Results/GroupByPropertyID/2015/'+ temp[i] +'.xlsx')

In [ ]:
#graph plotting

In [19]:
# Our numerical workhorses
import numpy as np
import pandas as pd


# Import Bokeh modules for interactive plotting
import bokeh.charts
import bokeh.charts.utils
import bokeh.io
import bokeh.models
import bokeh.palettes
import bokeh.plotting

# Display graphics in this notebook
bokeh.io.output_notebook()

Loading BokehJS ...

In [20]:
import bokeh.plotting as plt
from bokeh.plotting import figure, output_file, show, ColumnDataSource
from bokeh.models import HoverTool
from bokeh.models.formatters import DatetimeTickFormatter

In [24]:
property_id_BB = pd.read_excel('Results/GroupByPropertyID/2015/BB.xlsx')
property_id_BB = property_id_BB.reset_index()
property_id_BB = property_id_BB.sort_values(by = 'Stay_date', ascending=True)
property_id_BB1 = property_id_BB.values.T.tolist()

In [33]:
property_id_BB[['ADR']].mean()

ADR    7850.974675
dtype: float64

In [34]:
property_id_BB[['totalRevenue']].mean()

totalRevenue    127449.844023
dtype: float64

In [35]:
property_id_BB[['totalRevenue']].sum()

totalRevenue    11088136.43
dtype: float64

In [23]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BB1[2],
            y=property_id_BB1[4],
            desc=property_id_BB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Room count","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for BB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [36]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BB1[2],
            y=property_id_BB1[5],
            desc=property_id_BB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for BB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [37]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BB1[2],
            y=property_id_BB1[6],
            desc=property_id_BB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for BB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [25]:
property_id_BB16 = pd.read_excel('C:/Users/Amulya/Desktop/EventsDataAnalysis/Task2/Results/GroupByPropertyID/2016/BB.xlsx')
property_id_BB16 = property_id_BB16.reset_index()
property_id_BB16 = property_id_BB16.sort_values(by = 'Stay_date', ascending=True)
property_id_BB116 = property_id_BB16.values.T.tolist()

In [26]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BB1[2],
            y=property_id_BB1[4],
            desc=property_id_BB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Stay_date']]
        )
    )

sourceB = ColumnDataSource(
        data=dict(
            x=property_id_BB116[2],
            y=property_id_BB116[4],
            desc=property_id_BB116[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB16['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Room Count","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs Room Count for BB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Room Count")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source, color ="blue", legend="2015") 
p.line('x', 'y', line_width=2, source=sourceB,color ="green", legend="2016") 
show(p)

In [40]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BB1[2],
            y=property_id_BB1[5],
            desc=property_id_BB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Stay_date']]
        )
    )

sourceB = ColumnDataSource(
        data=dict(
            x=property_id_BB116[2],
            y=property_id_BB116[5],
            desc=property_id_BB116[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB16['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for BB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source, color ="blue", legend="2015") 
p.line('x', 'y', line_width=2, source=sourceB,color ="green", legend="2016") 
show(p)

In [41]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BB1[2],
            y=property_id_BB1[6],
            desc=property_id_BB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB['Stay_date']]
        )
    )

sourceA = ColumnDataSource(
        data=dict(
            x=property_id_BB116[2],
            y=property_id_BB116[6],
            desc=property_id_BB116[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BB16['Stay_date']]
        )
    )


hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for BB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source,color ="blue", legend="2015") 
p.line('x', 'y', line_width=2, source=sourceA,color ="green", legend="2016") 
show(p)

In [ ]:
property_id_BG = pd.read_excel('Results/GroupByPropertyID/2015/BG.xlsx')
property_id_BG = property_id_BG.reset_index()
property_id_BG = property_id_BG.sort_values(by = 'Stay_date', ascending=True)
property_id_BG1 = property_id_BG.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BG1[2],
            y=property_id_BG1[5],
            desc=property_id_BG1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BG['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for BG (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_BG1[2],
            y=property_id_BG1[6],
            desc=property_id_BG1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_BG['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for BG (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_CH = pd.read_excel('Results/GroupByPropertyID/2015/CH.xlsx')
property_id_CH = property_id_CH.reset_index()
property_id_CH = property_id_CH.sort_values(by = 'Stay_date', ascending=True)
property_id_CH1 = property_id_CH.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CH1[2],
            y=property_id_CH1[5],
            desc=property_id_CH1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CH['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for CH (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CH1[2],
            y=property_id_CH1[6],
            desc=property_id_CH1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CH['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for CH (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_CL = pd.read_excel('Results/GroupByPropertyID/2015/CL.xlsx')
property_id_CL = property_id_CL.reset_index()
property_id_CL = property_id_CL.sort_values(by = 'Stay_date', ascending=True)
property_id_CL1 = property_id_CL.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CL1[2],
            y=property_id_CL1[5],
            desc=property_id_CL1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CL['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for CL (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CL1[2],
            y=property_id_CL1[6],
            desc=property_id_CL1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CL['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for CL (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_CP = pd.read_excel('Results/GroupByPropertyID/2015/CP.xlsx')
property_id_CP = property_id_CP.reset_index()
property_id_CP = property_id_CP.sort_values(by = 'Stay_date', ascending=True)
property_id_CP1 = property_id_CP.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CP1[2],
            y=property_id_CP1[5],
            desc=property_id_CP1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CP['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for CP (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_CP1[2],
            y=property_id_CP1[6],
            desc=property_id_CP1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_CP['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for CP (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_EB = pd.read_excel('Results/GroupByPropertyID/2015/EB.xlsx')
property_id_EB = property_id_EB.reset_index()
property_id_EB = property_id_EB.sort_values(by = 'Stay_date', ascending=True)
property_id_EB1 = property_id_EB.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_EB1[2],
            y=property_id_EB1[5],
            desc=property_id_EB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_EB['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for EB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_EB1[2],
            y=property_id_EB1[6],
            desc=property_id_EB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_EB['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for EB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_KK = pd.read_excel('Results/GroupByPropertyID/2015/KK.xlsx')
property_id_KK = property_id_KK.reset_index()
property_id_KK = property_id_KK.sort_values(by = 'Stay_date', ascending=True)
property_id_KK1 = property_id_KK.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_KK1[2],
            y=property_id_KK1[5],
            desc=property_id_KK1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_KK['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for KK (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_KK1[2],
            y=property_id_KK1[6],
            desc=property_id_KK1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_KK['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for KK (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_KR = pd.read_excel('Results/GroupByPropertyID/2015/KR.xlsx')
property_id_KR = property_id_KR.reset_index()
property_id_KR = property_id_KR.sort_values(by = 'Stay_date', ascending=True)
property_id_KR1 = property_id_KR.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_KR1[2],
            y=property_id_KR1[5],
            desc=property_id_KR1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_KR['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for KR (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_KR1[2],
            y=property_id_KR1[6],
            desc=property_id_KR1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_KR['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for KR (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_MB = pd.read_excel('Results/GroupByPropertyID/2015/MB.xlsx')
property_id_MB = property_id_MB.reset_index()
property_id_MB = property_id_MB.sort_values(by = 'Stay_date', ascending=True)
property_id_MB1 = property_id_MB.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MB1[2],
            y=property_id_MB1[5],
            desc=property_id_MB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MB['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for MB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MB1[2],
            y=property_id_MB1[6],
            desc=property_id_MB1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MB['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for MB (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_MM = pd.read_excel('Results/GroupByPropertyID/2015/MM.xlsx')
property_id_MM = property_id_MM.reset_index()
property_id_MM = property_id_MM.sort_values(by = 'Stay_date', ascending=True)
property_id_MM1 = property_id_MM.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MM1[2],
            y=property_id_MM1[5],
            desc=property_id_MM1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MM['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for MM (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_MM1[2],
            y=property_id_MM1[6],
            desc=property_id_MM1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_MM['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for MM (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_SC = pd.read_excel('Results/GroupByPropertyID/2015/SC.xlsx')
property_id_SC = property_id_SC.reset_index()
property_id_SC = property_id_SC.sort_values(by = 'Stay_date', ascending=True)
property_id_SC1 = property_id_SC.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SC1[2],
            y=property_id_SC1[5],
            desc=property_id_SC1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SC['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for SC (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SC1[2],
            y=property_id_SC1[6],
            desc=property_id_SC1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SC['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for SC (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_SS = pd.read_excel('Results/GroupByPropertyID/2015/SS.xlsx')
property_id_SS = property_id_SS.reset_index()
property_id_SS = property_id_SS.sort_values(by = 'Stay_date', ascending=True)
property_id_SS1 = property_id_SS.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SS1[2],
            y=property_id_SS1[5],
            desc=property_id_SS1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SS['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for SS (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SS1[2],
            y=property_id_SS1[6],
            desc=property_id_SS1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SS['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for SS (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_SV = pd.read_excel('Results/GroupByPropertyID/2015/SV.xlsx')
property_id_SV = property_id_SV.reset_index()
property_id_SV = property_id_SV.sort_values(by = 'Stay_date', ascending=True)
property_id_SV1 = property_id_SV.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SV1[2],
            y=property_id_SV1[5],
            desc=property_id_SV1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SV['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for SV (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_SV1[2],
            y=property_id_SV1[6],
            desc=property_id_SV1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_SV['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for SV (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_VM = pd.read_excel('Results/GroupByPropertyID/2015/VM.xlsx')
property_id_VM = property_id_VM.reset_index()
property_id_VM = property_id_VM.sort_values(by = 'Stay_date', ascending=True)
property_id_VM1 = property_id_VM.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_VM1[2],
            y=property_id_VM1[5],
            desc=property_id_VM1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_VM['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for VM (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_VM1[2],
            y=property_id_VM1[6],
            desc=property_id_VM1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_VM['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for VM (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
property_id_VV = pd.read_excel('Results/GroupByPropertyID/2015/VV.xlsx')
property_id_VV = property_id_VV.reset_index()
property_id_VV = property_id_VV.sort_values(by = 'Stay_date', ascending=True)
property_id_VV1 = property_id_VV.values.T.tolist()

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_VV1[2],
            y=property_id_VV1[5],
            desc=property_id_VV1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_VV['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("Total Revenue","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs TotalRevenue for VV (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "Total Revenue")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)

In [ ]:
source = ColumnDataSource(
        data=dict(
            x=property_id_VV1[2],
            y=property_id_VV1[6],
            desc=property_id_VV1[3],
            date=[x.strftime("%Y-%m-%d") for x in property_id_VV['Stay_date']]
        )
    )

hover = HoverTool(
        tooltips=[
            ("Date","@date"),
            ("Event", "@desc"),
            ("ADR","@y{1.11}")
        ]
    )

p = plt.figure(x_axis_type="datetime",width=800, height=500, tools=[hover],
           title="Graph of StayDate vs ADR for VV (hover to view events)",x_axis_label = "Stay Date",
               y_axis_label = "ADR")
p.left[0].formatter.use_scientific = False
p.line('x', 'y', line_width=2, source=source)   
show(p)